# TruBot testing

This notebook tests a conversation bot with vector-store context of TruEra website. 

In [ ]:
%load_ext autoreload
%autoreload 2
from pathlib import Path
import sys

sys.path.append(str(Path().cwd().parent.parent.resolve()))

# Uncomment to get more debugging printouts:
"""
import logging

root = logging.getLogger()
root.setLevel(logging.DEBUG)

handler = logging.StreamHandler(sys.stdout)
handler.setLevel(logging.DEBUG)
formatter = logging.Formatter('%(asctime)s - %(name)s - %(levelname)s - %(message)s')
handler.setFormatter(formatter)
root.addHandler(handler)
"""
None

In [ ]:
from trulens_eval.keys import check_or_set_keys

check_or_set_keys(
    OPENAI_API_KEY="to fill in",
    HUGGINGFACE_API_KEY="to fill in",
    PINECONE_API_KEY="to fill in",
    PINECONE_ENV="to fill in"
)

In [ ]:
from examples.trubot.trubot import get_or_make_app, get_answer, f_lang_match, f_qs_relevance
from trulens_eval.util import TP
from trulens_eval import Tru

from pprint import PrettyPrinter
pp = PrettyPrinter()

# Reset the database if needed:
Tru().reset_database()

In [ ]:
app = get_or_make_app(cid=None)

In [ ]:
# List the app components that got instrumented along with their categories.

app.print_instrumented()

In [ ]:
proc = Tru().start_dashboard(force=True, _dev=Path.cwd().parent.parent)

In [ ]:
selectors = [0,1,3,4]
messages = ["Who is Shayak?", "Wer ist Shayak?", "Kim jest Shayak?", "¿Quién es Shayak?", "Was ist QII?", "Co jest QII?"]

selectors = selectors[2:3]
messages = messages[2:3]

def test_bot(selector, question):
    print(selector, question)
    app = get_or_make_app(cid=question + str(selector), selector=selector)
    answer = get_answer(app=app, question=question)
    return answer

results = []

for s in selectors:
    for m in messages:
        results.append(TP().promise(test_bot, selector=s, question=m))

In [ ]:
thread = Tru().start_evaluator(restart=True)